In [2]:
import sys, re
import xml.etree.ElementTree as ET

In [ ]:
def guess_prop_cats(t, s): 
    cats = []
    for c in s: 
        if c.count('Sostinė') > 0: cats.append('LOC') # столица
        if c.count('Miestas') > 0: cats.append('LOC') # города
        if c.count('miestai') > 0: cats.append('LOC') # города
        if c.count('Žmonės') > 0: cats.append('PER') # люди
        if c.count('Šalys') > 0: cats.append('LOC') # страны
        if c.count('šalys') > 0: cats.append('LOC') # страны
        if c.count('provincijos') > 0: cats.append('LOC') # провинции
        if c.count('Kaimai') > 0: cats.append('LOC') # деревни
        if t.split(' ')[-1] == 'provincijos': cats.append('LOC')  # провинции
        if t.split(' ')[-1] == 'provincijos)': cats.append('LOC')  # провинции
        return list(set(cats))
    
doc = ET.parse('ltwiki-20171220-pages-articles-multistream.xml')
ns = {'mediawiki': 'http://www.mediawiki.org/xml/export-0.10/'}
cats = re.compile('(\[\[Kategorija:[^|\]]+\]\]|\[\[Category:[^|\]]+\]\])')

# For each of the pages
for p in doc.findall('.//mediawiki:page', ns): 
    # Find the title
    t = p.findall('.//mediawiki:title', ns)[0].text

    # If the title is outside the main namespace
    if t.count(':') > 0: continue
    # If the title only consists of numerals
    if re.match('^[0-9]+$', t): continue;	

    # Pull out the text
    tekst = str(p.findall('.//mediawiki:text', ns)[0].text)

    # Find all the categories
    incat = cats.findall(tekst)
    # Classify the categories into named-entities
    pars = guess_prop_cats(t, incat)
    if pars != []:
        print(t + '\t' + '/'.join(pars) + '\t' + '/'.join(incat))